In [ ]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import *

### Read in Source Data and Create 3 Scenarios

In [ ]:
df = pd.read_csv("./data/TSPTW_150.txt")

In [ ]:
all_vals = []
for i in range(df.shape[0]):
    vals = [val for val in df.iloc[i,0].strip().split(" ") if val != ""]
    all_vals.append(vals)

df_data = pd.DataFrame(all_vals, columns = ["CUST NO.", "X", "Y", "DEMAND", "READY TIME", "DUE DATE", "SERVICE TIME"])
df_data = df_data.iloc[:-1,:]
df_data["COORDINATES"] = df_data.apply(lambda row: (row["X"], row["Y"]), axis=1)
df_data["DEMAND"] = np.random.randint(1,5,df_data.shape[0])
df_data["SERVICE TIME"] = np.random.randint(1,9,df_data.shape[0])
df_data = df_data.drop(columns=["X", "Y"])

In [ ]:
df_small = df_data.iloc[1:].sample(n = 80, replace = False, random_state=42).copy()
df_medium = df_data.iloc[1:].sample(n = 100, replace = False, random_state=42).copy()
df_large = df_data.iloc[1:].sample(n = 120, replace = False, random_state=42).copy()

In [ ]:
initial = pd.DataFrame([df_data.iloc[0].to_list()], columns = df_data.columns)

In [ ]:
df_small = pd.concat([initial, df_small]).reset_index(drop=True)
df_medium = pd.concat([initial, df_medium]).reset_index(drop=True)
df_large = pd.concat([initial, df_large]).reset_index(drop=True)

### Create Distance and Time Matrices

In [ ]:
import math
# Compute pairwise times matrix

def distance(df, col, row1, row2):
    c1 = df.loc[row1, col]
    c2 = df.loc[row2, col]
    diff = (float(c1[0])-float(c2[0]), float(c1[1])-float(c2[1]))
    return math.sqrt(diff[0]*diff[0]+diff[1]*diff[1])


In [ ]:
small_dist = []
for row1 in range(df_small.shape[0]):
    row_vals = []
    for row2 in range(df_small.shape[0]):
        row_vals.append(distance(df_small, "COORDINATES", row1, row2))
    small_dist.append(row_vals)

In [ ]:
medium_dist = []
for row1 in range(df_medium.shape[0]):
    row_vals = []
    for row2 in range(df_medium.shape[0]):
        row_vals.append(distance(df_medium, "COORDINATES", row1, row2))
    medium_dist.append(row_vals)

In [ ]:
large_dist = []
for row1 in range(df_large.shape[0]):
    row_vals = []
    for row2 in range(df_large.shape[0]):
        row_vals.append(distance(df_large, "COORDINATES", row1, row2))
    large_dist.append(row_vals)

In [ ]:
dist_small = np.asarray(small_dist)
dist_medium = np.asarray(medium_dist)
dist_large = np.asarray(large_dist)

### Model Implementation - Small

In [ ]:
I = len(dist_small)
J = len(dist_small[0])

# Start Times
A = df_small["READY TIME"].astype(float).to_numpy()

# End Times
B = df_small["DUE DATE"].astype(float).to_numpy()

# distance matrix
D = dist_small

In [ ]:
probS = gp.Model("TSPTW - Concerts - Small")

X = probS.addVars(I, J, vtype=GRB.BINARY, name=[f"({i}), ({j})" for i in range(I) for j in range(J)])
TI = probS.addVars(I, vtype=GRB.INTEGER, name=[f"TI_{i}" for i in range(I)])
TJ = probS.addVars(J, vtype=GRB.INTEGER, name=[f"TJ_{j}" for j in range(J)])

probS.setObjective(gp.quicksum(X[i,j]*D[i][j] for i in range(I) for j in range(J)), GRB.MINIMIZE)

In [ ]:
for i in range(I):
    # Initiation of Arrival to first node
    probS.addConstr(TI[i] - D[0][i]*X[0,i] >= 0)

    # start of time window
    probS.addConstr(TI[i] >= A[i])

    # end of time window
    probS.addConstr(TJ[i] <= B[i])

    # only one per row
    probS.addConstr(gp.quicksum(X[i,j] for j in range(J)) == 1, name = "row" + str(i))

    # cannot go along diagonals
    probS.addConstr(X[i,i] == 0)

for j in range(J):
    # only one per column
    probS.addConstr(gp.quicksum(X[i,j] for i in range(I)) == 1, name = "col" + str(j))

In [ ]:
# Sub tour Elimination
for i in range(I):
    for j in range(J):
        if i == j:
            continue
        else:
            probS.addConstr(TI[i] - TJ[j] + (B[i] - A[j] + D[i][j])*X[i,j] <= B[i] - A[j], name = "row" + str(i) + " - col" + str(j))

In [ ]:
probS.optimize()

In [ ]:
tot = 0
for v in probS.getVars():
    if v.x > 0:
        print(v.varName, v.x, v.Obj)
        tot += v.Obj

### Model Implementation - Medium

In [ ]:
I = len(dist_medium)
J = len(dist_medium[0])

# Start Times
A = df_medium["READY TIME"].astype(float).to_numpy()

# End Times
B = df_medium["DUE DATE"].astype(float).to_numpy()

# distance matrix
D = dist_medium

In [ ]:
probM = gp.Model("TSPTW - Concerts - Medium")

X = probM.addVars(I, J, vtype=GRB.BINARY, name=[f"({i}), ({j})" for i in range(I) for j in range(J)])
TI = probM.addVars(I, vtype=GRB.INTEGER, name=[f"TI_{i}" for i in range(I)])
TJ = probM.addVars(J, vtype=GRB.INTEGER, name=[f"TJ_{j}" for j in range(J)])

probM.setObjective(gp.quicksum(X[i,j]*D[i][j] for i in range(I) for j in range(J)), GRB.MINIMIZE)

In [ ]:
for i in range(I):
    # Initiation of Arrival to first node
    probM.addConstr(TI[i] - D[0][i]*X[0,i] >= 0)

    # start of time window
    probM.addConstr(TI[i] >= A[i])

    # end of time window
    probM.addConstr(TJ[i] <= B[i])

    # only one per row
    probM.addConstr(gp.quicksum(X[i,j] for j in range(J)) == 1, name = "row" + str(i))

    # cannot go along diagonals
    probM.addConstr(X[i,i] == 0)

for j in range(J):
    # only one per column
    probM.addConstr(gp.quicksum(X[i,j] for i in range(I)) == 1, name = "col" + str(j))

In [ ]:
# Sub tour Elimination
for i in range(I):
    for j in range(J):
        if i == j:
            continue
        else:
            probM.addConstr(TI[i] - TJ[j] + (B[i] - A[j] + D[i][j])*X[i,j] <= B[i] - A[j], name = "row" + str(i) + " - col" + str(j))

In [ ]:
probM.optimize()

In [ ]:
tot = 0
for v in probM.getVars():
    if v.x > 0:
        print(v.varName, v.x, v.Obj)
        tot += v.Obj

### Model Implementation - Large

In [ ]:
I = len(dist_large)
J = len(dist_large[0])

# Start Times
A = df_large["READY TIME"].astype(float).to_numpy()

# End Times
B = df_large["DUE DATE"].astype(float).to_numpy()

# distance matrix
D = dist_large

In [ ]:
probL = gp.Model("TSPTW - Concerts - Medium")

X = probL.addVars(I, J, vtype=GRB.BINARY, name=[f"({i}), ({j})" for i in range(I) for j in range(J)])
TI = probL.addVars(I, vtype=GRB.INTEGER, name=[f"TI_{i}" for i in range(I)])
TJ = probL.addVars(J, vtype=GRB.INTEGER, name=[f"TJ_{j}" for j in range(J)])

probL.setObjective(gp.quicksum(X[i,j]*D[i][j] for i in range(I) for j in range(J)), GRB.MINIMIZE)

In [ ]:
for i in range(I):
    # Initiation of Arrival to first node
    probL.addConstr(TI[i] - D[0][i]*X[0,i] >= 0)

    # start of time window
    probL.addConstr(TI[i] >= A[i])

    # end of time window
    probL.addConstr(TJ[i] <= B[i])

    # only one per row
    probL.addConstr(gp.quicksum(X[i,j] for j in range(J)) == 1, name = "row" + str(i))

    # cannot go along diagonals
    probL.addConstr(X[i,i] == 0)

for j in range(J):
    # only one per column
    probL.addConstr(gp.quicksum(X[i,j] for i in range(I)) == 1, name = "col" + str(j))

In [ ]:
# Sub tour Elimination
for i in range(I):
    for j in range(J):
        if i == j:
            continue
        else:
            probL.addConstr(TI[i] - TJ[j] + (B[i] - A[j] + D[i][j])*X[i,j] <= B[i] - A[j], name = "row" + str(i) + " - col" + str(j))

In [ ]:
probL.optimize()

In [ ]:
tot = 0
for v in probL.getVars():
    if v.x > 0:
        print(v.varName, v.x, v.Obj)
        tot += v.Obj